In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from sklearn.metrics import mean_squared_error, r2_score 
%matplotlib inline

In [2]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sample = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [3]:
# copy train & test data
train_df = train
test_df = test

In [4]:
sample.head()

,id,pressure
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [5]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [6]:
print(train.shape)
print(test.shape)

(6036000, 8)
(4024000, 7)


In [7]:
test.head()

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0


In [8]:
print(train.isnull().sum())
print(test.isnull().sum())

id           0
breath_id    0
R            0
C            0
time_step    0
u_in         0
u_out        0
pressure     0
dtype: int64
id           0
breath_id    0
R            0
C            0
time_step    0
u_in         0
u_out        0
dtype: int64


In [9]:
train.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,2.703618e+01,2.608072e+01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01
std,1.742443e+06,3.633526e+04,1.959549e+01,1.715231e+01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00
25%,1.509001e+06,3.137700e+04,5.000000e+00,1.000000e+01,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00
50%,3.018000e+06,6.276550e+04,2.000000e+01,2.000000e+01,1.308123e+00,4.386146e+00,1.000000e+00,7.032628e+00
75%,4.527000e+06,9.430100e+04,5.000000e+01,5.000000e+01,1.965502e+00,4.983895e+00,1.000000e+00,1.364103e+01
max,6.036000e+06,1.257490e+05,5.000000e+01,5.000000e+01,2.937238e+00,1.000000e+02,1.000000e+00,6.482099e+01


In [10]:
print(train.shape)
print(train.columns)

(6036000, 8)
Index(['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure'], dtype='object')


In [11]:
train = train.drop(['id','breath_id'], axis = 1)
test = test.drop(['id','breath_id'], axis = 1)
X=train.drop('pressure',axis=1)
y = train['pressure']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, random_state=101, test_size=0.1)

In [13]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [14]:
X_train_std= sc.fit_transform(X_train)
test_std= sc.transform(test)
X_val_std= sc.transform(X_val)

In [15]:
X_train_std

array([[-1.12450265, -0.93764418, -0.35991722, -0.54498622,  0.78207629],
       [-1.12450265, -0.35465345,  1.45515061, -0.17597426,  0.78207629],
       [-0.35901541, -0.35465345,  0.20387996, -0.54498622,  0.78207629],
       ...,
       [-1.12450265,  1.39431875,  1.80491167, -0.1736116 ,  0.78207629],
       [-0.35901541, -0.93764418, -0.25469924, -0.54498622,  0.78207629],
       [-0.35901541, -0.93764418,  1.77536654, -0.17382574,  0.78207629]])

In [16]:
X_val_std

array([[ 1.17195909, -0.35465345,  1.08676907, -0.18574118,  0.78207629],
       [-0.35901541, -0.35465345, -1.39571244, -0.03461089, -1.27864764],
       [-1.12450265, -0.35465345, -1.24965801,  0.0495652 , -1.27864764],
       ...,
       [ 1.17195909, -0.93764418, -0.29084827, -0.37752119,  0.78207629],
       [-1.12450265, -0.35465345, -0.99809611,  1.31591638, -1.27864764],
       [-1.12450265, -0.93764418, -0.53597093, -0.42872924, -1.27864764]])

In [17]:
test_std

array([[-1.12450265, -0.35465345, -1.70683524, -0.54498622, -1.27864764],
       [-1.12450265, -0.35465345, -1.66518295,  0.01444264, -1.27864764],
       [-1.12450265, -0.35465345, -1.62350483,  0.54570178, -1.27864764],
       ...,
       [-0.35901541, -0.93764418,  1.6843269 , -0.17430878,  0.78207629],
       [-0.35901541, -0.93764418,  1.72827045, -0.17407203,  0.78207629],
       [-0.35901541, -0.93764418,  1.77287793, -0.17386918,  0.78207629]])

In [18]:
y_train

4274590    14.625260
3684476     6.329607
5601723     5.696887
4569970     5.415679
4521169     6.540513
             ...    
5713995     6.681117
311895      6.962326
4855359     6.189002
1525073     8.508973
3322719     6.470211
Name: pressure, Length: 5432400, dtype: float64

In [19]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()

In [20]:
lr.fit(X_train_std,y_train)

LinearRegression()

In [21]:
Y_pred_lr=lr.predict(X_val_std)

In [22]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(r2_score(y_val,Y_pred_lr))
print(mean_absolute_error(y_val,Y_pred_lr))
print(np.sqrt(mean_squared_error(y_val,Y_pred_lr)))

0.3830744254605081
3.9541869553909783
6.367650936698984


Neural Network

In [23]:
# Initialising the ANN
model= Sequential()

# Adding the input layer
model.add(Dense(units=64,activation='relu',input_shape=[5]))
# model.add(Dense(units=64,activation='relu',input_shape=[len(X_train.keys())]))

# Adding the second hidden layer
model.add(Dense(units=64,activation='relu'))

# Adding the 3rd hidden layer
model.add(Dense(units=64,activation='relu'))

# Adding the output layer
model.add(Dense(units=1))


User settings:

   KMP_AFFINITY=granularity=fine,noverbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER

In [24]:
model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['mean_squared_error','mean_absolute_error'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                384       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 8,769
Trainable params: 8,769
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(X_train_std,y_train,batch_size=100,epochs=10)

2022-01-31 13:33:15.735201: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
54324/54324 [==============================] - 155s 3ms/step - loss: 18.7129 - mean_squared_error: 18.7129 - mean_absolute_error: 2.2599
Epoch 2/10
54324/54324 [==============================] - 155s 3ms/step - loss: 17.6316 - mean_squared_error: 17.6316 - mean_absolute_error: 2.1604
Epoch 3/10
54324/54324 [==============================] - 152s 3ms/step - loss: 17.4541 - mean_squared_error: 17.4541 - mean_absolute_error: 2.1454
Epoch 4/10
54324/54324 [==============================] - 153s 3ms/step - loss: 17.3473 - mean_squared_error: 17.3473 - mean_absolute_error: 2.1364
Epoch 5/10
54324/54324 [==============================] - 156s 3ms/step - loss: 17.2833 - mean_squared_error: 17.2833 - mean_absolute_error: 2.1316
Epoch 6/10
54324/54324 [==============================] - 155s 3ms/step - loss: 17.2198 - mean_squared_error: 17.2198 - mean_absolute_error: 2.1258
Epoch 7/10
54324/54324 [==============================] - 155s 3ms/step - loss: 17.1841 - mean_squared_error: 17

In [27]:
mae=model.evaluate(X_train_std,y_train)

169763/169763 [==============================] - 306s 2ms/step - loss: 17.0651 - mean_squared_error: 17.0651 - mean_absolute_error: 2.1118


In [28]:
prediction=model.predict(test_std)

In [29]:
prediction = pd.DataFrame(prediction)

In [30]:
prediction

,0
0,5.996200
1,6.333659
2,7.375490
3,8.444229
4,9.935358
...,...
4023995,6.151299
4023996,6.129362
4023997,6.106656
4023998,6.081875
